<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment01/CS6910_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: karthik21 (use `wandb login --relogin` to force relogin)


True

#Question 1 (2 Marks)
Download the fashion-MNIST dataset and plot 1 sample image for each class as shown in the grid below. Use "from keras.datasets import fashion_mnist" for getting the fashion mnist dataset.

﻿

In [ ]:
from keras.datasets import fashion_mnist
from matplotlib import pyplot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
label = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for i in range(10):
   wandb.init(project="cs6910-assignment01")
   wandb.run.name = "Q1_run_{}".format(i+1)
   for j in range(30):
     if y_train[j] == i:
       wandb.log({"examples": [wandb.Image(X_train[j], caption=label[i])]})
       break

# Question 2 (10 Marks)

Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

Your code should be flexible so that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.

In [ ]:
import numpy as np


def sigmoid(z):
  g = 1/(1+np.exp(-z))
  return g

def tanh(z):
  g = np.tanh(z)
  return g

def relu(z):
  return np.maximum(0,z)

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
def initialize(initializer, size1, size2):
  if(initializer == "random"):
     W = np.random.randn(size1, size2) * 0.01
     return W

  if(initializer == "Xavier"):
     W = np.random.randn(size1, size2) * np.sqrt(1/size2)
     return W

  print("Enter the name of initializer correctly")

In [ ]:
def linear_forward(H, W, b):
  W = np.asarray(W)
  H = np.reshape(H,(H.shape[0],-1))
  A = np.dot(W,H) + b 
  cache = (H, W, b)
  
  return A, cache

In [ ]:
def initialize_Wb_matrix(X, num_hidden, size_hidden, initializer):
  layer_dims = [X.shape[0]]
  for l in range(0, num_hidden):
    layer_dims.append(size_hidden)
  layer_dims.append(10)   
  np.random.seed(3)
  Wb_matrix = {}
  update = {}
  grads = {}
  L = len(layer_dims)            # number of layers in the network

  for l in range(1,L):
    Wb_matrix['W' + str(l)] = initialize(initializer, layer_dims[l], layer_dims[l-1])
    update['W' + str(l)] = np.zeros((layer_dims[l], layer_dims[l-1]))
    Wb_matrix['b' + str(l)] = np.zeros((layer_dims[l], 1))
    update['b' + str(l)] = np.zeros((layer_dims[l], 1))
    grads['dW' + str(l)] = np.zeros((layer_dims[l], layer_dims[l-1]))
    grads['db' + str(l)] = np.zeros((layer_dims[l], 1))
  return Wb_matrix, update, grads

In [ ]:
def activation_forward(H_prev, W, b, activation):
  A, linear_cache = linear_forward(H_prev, W, b)
  activation_cache = A
  if activation == 'relu':
    H = relu(A)
  elif activation == 'sigmoid':
    H = sigmoid(A)
  elif activation =='tanh':
    H = tanh(A)
  elif activation == 'softmax':
    H = softmax(A)
  
  return H, activation_cache, linear_cache

In [ ]:
def forward_propagation(X, Wb_matrix, activation):
    H = X
    L = int((len(Wb_matrix)/2))
    A_caches = []
    H_caches = [H]
    for l in range(1, L):
        H_prev = H 
        H, A_cache, linear_cache = activation_forward(H_prev, Wb_matrix['W{:d}'.format(l)], Wb_matrix['b{:d}'.format(l)], activation)
        A_caches.append(A_cache)
        H_caches.append(H)
    HL, AL, linear_cache = activation_forward(H, Wb_matrix['W%d' % L], Wb_matrix['b%d' % L], activation='softmax')
    A_caches.append(AL)
    H_caches.append(HL)
    return HL, H_caches, A_caches

# Question 3 (18 Marks)

Implement the backpropagation algorithm with support for the following optimisation functions 

- sgd
- momentum based gradient descent
- nesterov accelerated gradient descent
- rmsprop
- adam
- nadam

(12 marks for the backpropagation framework and 2 marks for each of the optimisation algorithms above)

We will check the code for implementation and ease of use (e.g., how easy it is to add a new optimisation algorithm such as Eve). Note that the code should be flexible enough to work with different batch sizes.

In [ ]:
def one_hot_encoding(l, L):
  import numpy
  e = []
  for i in range(L):
    if i == l:
      e.append(1)
    else:
      e.append(0)
  e_y = np.asarray(e)
  return e_y

In [ ]:
def deriv_activ(z, activation):
  if(activation == "relu"):
    z[z<=0] = 0
    z[z>0] = 1
    return z

  elif(activation == "sigmoid"):
    g_deriv = sigmoid(z) * (1 - sigmoid(z))
    return g_deriv
  
  elif(activation == "tanh"):
    deriv = 1 - (tanh(z)) ** 2
    return deriv

In [ ]:
def back_propagation(H_caches, A_caches, Wb_matrix, y_hat, y, activation, weight_decay,loss):
  e_y = []
  L = int((len(Wb_matrix)/2))
  num_train = y_hat.shape[1]
  num_classes = y_hat.shape[0]
  for j in range(num_train):
    e_y.append(one_hot_encoding(y[j],num_classes))
  e_y = np.reshape(e_y,(num_classes,num_train))
  if loss == "cross_entropy":
    da = -(e_y - y_hat)
  elif loss == "mse":
    da = (y_hat - e_y)*y_hat - y_hat*(np.dot((y_hat-y).T, y_hat))
  da = -(e_y - y_hat)
  grads = {}
  
  for k in reversed(range(1,L+1)):
    dW = np.matmul(da, np.transpose(H_caches[k-1])) + 2*weight_decay*Wb_matrix['W{:d}'.format(k)]
    db = da
    if k != 1:
      dh = np.dot(Wb_matrix['W{:d}'.format(k)].T, da)
      da = np.multiply(dh, deriv_activ(A_caches[k-2], activation))
    grads["dW" + str(k)] = dW
    grads["db" + str(k)] = db
  
  return grads

In [ ]:
def sgd(Wb_matrix, grads, eta,L):
  for l in range(L):
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - eta * grads["dW" + str(l+1)]
    Wb_matrix['b' + str(l+1)] = Wb_matrix['b' + str(l+1)] - eta * grads["db" + str(l+1)]
  return Wb_matrix

def momentum(Wb_matrix, grads, eta, gamma, update,L):
  for l in range(L):
    update['W' + str(l+1)] = (gamma * update['W' + str(l+1)]) + eta * grads['dW' + str(l+1)]
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - update['W' + str(l+1)]

    update["b" + str(l+1)] = (gamma * update["b" + str(l+1)]) + eta * grads["db" + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update["b" + str(l+1)]
  
  return Wb_matrix, update

def nag(Wb_matrix,eta,gamma, update, L):
  Wb_matrix_temp= {}
  for l in range(L):
    update['W' + str(l+1)] = (gamma * update['W' + str(l+1)])
    Wb_matrix_temp['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - update['W' + str(l+1)]

    update["b" + str(l+1)] = (gamma * update["b" + str(l+1)])
    Wb_matrix_temp["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - update["b" + str(l+1)]

  return Wb_matrix_temp, update

def rmsprop(Wb_matrix, eta, update ,beta1, eps, grads, L):
  for l in range(L):
    update['W' + str(l+1)] = beta1 * update['W' + str(l+1)] + (1-beta1) * grads['dW' + str(l+1)]**2
    update["b" + str(l+1)] = beta1 * update["b" + str(l+1)] + (1-beta1) * grads["db" + str(l+1)]**2
    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - (eta/np.sqrt(update['W' + str(l+1)]+eps)) * grads['dW' + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - (eta/np.sqrt(update['b' + str(l+1)]+eps)) * grads['db' + str(l+1)]
  return Wb_matrix, update
  
def adam(Wb_matrix, eta, beta1, beta2, update, v_n, eps, grads, L,i):
  import math
  m_hat = update.copy()
  v_hat = update.copy()
  for l in range(L):
    update['W' + str(l+1)] = beta1 * update['W' + str(l+1)] + (1-beta1) * grads['dW' + str(l+1)]
    update['b' + str(l+1)] = beta1 * update['b' + str(l+1)] + (1-beta1) * grads['db' + str(l+1)]
    
    v_n['W' + str(l+1)] = beta2 * v_n['W' + str(l+1)] + (1-beta2) * grads['dW' + str(l+1)]**2
    v_n["b" + str(l+1)] = beta2 * v_n["b" + str(l+1)] + (1-beta2) * grads['db' + str(l+1)]**2
    
    m_hat['W' + str(l+1)] = update['W' + str(l+1)]/(1-math.pow(beta1,i+1))
    m_hat['b' + str(l+1)] = update['b' + str(l+1)]/(1-math.pow(beta1,i+1))

    v_hat['W' + str(l+1)] = v_n['W' + str(l+1)]/(1-math.pow(beta2,i+1))
    v_hat['b' + str(l+1)] = v_n['b' + str(l+1)]/(1-math.pow(beta2,i+1))


    Wb_matrix['W' + str(l+1)] = Wb_matrix['W' + str(l+1)] - (eta/(np.sqrt(v_hat['W' + str(l+1)]+eps)))* m_hat['W' + str(l+1)]
    Wb_matrix["b" + str(l+1)] = Wb_matrix["b" + str(l+1)] - (eta/(np.sqrt(v_hat['b' + str(l+1)]+eps)))* m_hat['b' + str(l+1)]

 
  return Wb_matrix, update, v_n

# Question 4 (10 Marks)

Use the sweep functionality provided by wandb to find the best values for the hyperparameters listed below. Use the standard train/test split of fashion_mnist (use (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()).  Keep 10% of the training data aside as validation data for this hyperparameter search. Here are some suggestions for different values to try for hyperparameters. As you can quickly see that this leads to an exponential number of combinations. You will have to think about strategies to do this hyperparameter search efficiently. Check out the options provided by wandb.sweep and write down what strategy you chose and why.

- number of epochs: 5, 10
- number of hidden layers:  3, 4, 5
- size of every hidden layer:  32, 64, 128
- weight decay (L2 regularisation): 0, 0.0005,  0.5
- learning rate: 1e-3, 1 e-4 
- optimizer:  sgd, momentum, nesterov, rmsprop, adam, nadam
- batch size: 16, 32, 64
- weight initialisation: random, Xavier
- activation functions: sigmoid, tanh, ReLU

wandb will automatically generate the following plots. Paste these plots below using the "Add Panel to Report" feature. Make sure you use meaningful names for each sweep (e.g. hl_3_bs_16_ac_tanh to indicate that there were 3 hidden layers, batch size was 16 and activation function was ReLU) instead of using the default names (whole-sweep, kind-sweep) given by wandb.

In [ ]:
def swp():
  from keras.datasets import fashion_mnist
  from matplotlib import pyplot
  import numpy as np
  import math
  import random
  from sklearn.metrics import confusion_matrix
  import seaborn as sns
  from sklearn.model_selection import train_test_split
  (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
  X_train_flatten = np.reshape(X_train,(X_train.shape[0],X_train.shape[1]*X_train.shape[2]))/255.0
  
  X_test_flatten = np.reshape(X_test,(X_test.shape[0],X_test.shape[1]*X_test.shape[2]))/255.0
  X_test_flatten = X_test_flatten.T

  X_train_flatten, X_val, y_train, y_val = train_test_split( X_train_flatten, y_train, test_size=0.1, random_state=42)
  X_train_flatten = X_train_flatten.T
  X_val = X_val.T
  class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
  num_train = int(0.9*X_train.shape[0])
  num_val = int(0.1*X_train.shape[0])
 
  hyperparameter_defaults = dict(
      num_hidden = 5,
      size_hidden = 128,
      weight_decay = 0,
      optimizer = 'adam',
      initializer = 'Xavier',
      activation = 'relu',
      batch_size = 64,
      learning_rate = 0.001,
      max_epoch = 10,
      loss_func = 'cross_entropy'
      )

  
  wandb.init(project="cs6910-assignment01", config=hyperparameter_defaults)
  config = wandb.config
  wandb.run.name = "hl{}_bs_{}_ac_{}".format(config.num_hidden,config.batch_size, config.activation)
  L = config.num_hidden+1
  beta1 = 0.9
  beta2 = 0.999
  eps = 1e-8
  gamma = 0.9


  Wb_matrix, update, grad_initial = initialize_Wb_matrix(X_train_flatten, config.num_hidden , config.size_hidden ,config.initializer)
  #initializing parameters in adam optimization to zero dictionary
  v = update.copy()
  m_hat = update.copy()
  v_hat = update.copy()

  
  for i in range(config.max_epoch):
    loss = 0
    val_loss = 0
    count = 0
    grads = grad_initial
    print("Epoch:", i+1)
    for j in range(num_train):
      train_ip= np.reshape(X_train_flatten[:,j], (-1, 1)) 
      y_hat, H_caches, A_caches = forward_propagation(train_ip, Wb_matrix ,config.activation)
      if(y_train[j]==np.argmax(y_hat)):
        count = count + 1; 

      if (j+1)%config.batch_size == 0  :
        if config.optimizer == 'nag':
          Wb_matrix, update = nag(Wb_matrix, config.learning_rate,gamma, update, L)
        elif config.optimizer == 'nadam':
          Wb_matrix, update = nag(Wb_matrix, config.learning_rate,gamma, update, L)

      grad = back_propagation(H_caches, A_caches, Wb_matrix, y_hat, [y_train[j]], config.activation, config.weight_decay,config.loss_func)
      for l in range(L):
        grads['dW' + str(l+1)] = grads['dW' + str(l+1)] + grad['dW' + str(l+1)] 
        grads['db' + str(l+1)] = grads['db' + str(l+1)] + grad['db' + str(l+1)] 


      if (j+1)%config.batch_size == 0 : 
        for l in range(L):
          grads['dW' + str(l+1)] = grads['dW' + str(l+1)]/config.batch_size
          grads['db' + str(l+1)] = grads['db' + str(l+1)]/config.batch_size
        if config.optimizer == 'sgd':
          Wb_matrix = sgd(Wb_matrix, grads, config.learning_rate,L)
        elif config.optimizer == 'momentum':
          Wb_matrix,update = momentum(Wb_matrix,grads,config.learning_rate,gamma,update,L)
        elif config.optimizer == 'nag':
          Wb_matrix,update = momentum(Wb_matrix,grads,config.learning_rate,gamma,update,L)
        elif config.optimizer == 'rmsprop':
          Wb_matrix, update = rmsprop(Wb_matrix, config.learning_rate, update,  beta1, eps, grads, L)
        elif config.optimizer == 'adam':
          Wb_matrix, update,v = adam(Wb_matrix, config.learning_rate, beta1, beta2, update, v, eps, grads, L, i+1)
        elif config.optimizer == 'nadam':
          Wb_matrix, update,v = adam(Wb_matrix, config.learning_rate, beta1, beta2, update, v, eps, grads, L, i+1)    
      if config.loss_func == 'cross_entropy':
        loss = loss - (1/num_train)*math.log(y_hat[y_train[j]])
      elif config.loss_func == 'mse':
        loss = loss + (1/num_train)*(max(y_hat) - y_train[j])**2
    
    y_hat_val, H_caches_val, A_caches_val = forward_propagation(X_val, Wb_matrix ,config.activation)
    count_val = np.sum(np.argmax(y_hat_val, axis = 0)== y_val)
    
    for j in range(num_val):
      val_ip= np.reshape(X_val[:,j], (-1, 1)) 
      y_hat_val, H_caches_val, A_caches_val = forward_propagation(val_ip, Wb_matrix ,config.activation)
      if config.loss_func == 'cross_entropy':
        val_loss = val_loss - (1/num_val)*math.log(y_hat_val[int(y_val[j])])
      elif config.loss_func == 'mse':
        val_loss = val_loss + (1/num_val)*(max(y_hat_val) - y_val[j])**2

    accuracy = 100*count/num_train
    val_accuracy = 100*count_val/num_val

    print("     Loss", loss)
    print("     Accuracy",accuracy)
    print("     Validation Loss", val_loss)
    print("     Validation Accuracy", val_accuracy)

    metrics = {'epoch':i, 'val_accuracy': val_accuracy, 'val_loss': val_loss, 'accuracy': accuracy, 'loss': loss}
    wandb.log(metrics)

  y_hat_test, H_caches_test, A_caches_test = forward_propagation(X_test_flatten, Wb_matrix ,config.activation)
  wandb.log({"Confusion_Matrix" : wandb.plot.confusion_matrix(
                        probs=None,
                        y_true=y_test,
                        preds=np.argmax(y_hat_test, axis = 0),
                        class_names=class_names)})
  

  wandb.run.finish()
   

In [ ]:
import wandb

sweep_config = {
  "name": "My Sweep",
  "method": "grid",
  "project": "cs6910-assignment01",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "max_epoch": {
            "values": [10]
        },
        "num_hidden": {
            "values":[5]
        }, 
        "size_hidden": {
            "values":[128]
        },
        "weight_decay":{
            "values":[0.0005]
        },
        "learning_rate":{
            "values":[0.001]
        },
        "batch_size": {
            "values":[16]
        },  
        "optimizer": {
            "values":['rmsprop', 'adam', 'nadam']
        },
        "initializer": {
            "values":['Xavier']
        },
        "activation":{
            "values": ['relu']
        },
        "loss_func":{
            "values": ['cross_entropy']
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: jrvbsjg2
Sweep URL: https://wandb.ai/karthik21/uncategorized/sweeps/jrvbsjg2


In [ ]:
wandb.agent(sweep_id, swp)

wandb: Agent Starting Run: skngemgr with config:
wandb: 	activation: relu
wandb: 	batch_size: 16
wandb: 	initializer: Xavier
wandb: 	learning_rate: 0.001
wandb: 	loss_func: cross_entropy
wandb: 	max_epoch: 10
wandb: 	num_hidden: 5
wandb: 	optimizer: rmsprop
wandb: 	size_hidden: 128
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch: 1
     Loss 0.5697750474974579
     Accuracy 79.22222222222223
     Validation Loss 0.5344589722873124
     Validation Accuracy 80.35
Epoch: 2
     Loss 0.45790988109401226
     Accuracy 83.78703703703704
     Validation Loss 0.4777744309036437
     Validation Accuracy 83.23333333333333
Epoch: 3
     Loss 0.4349588823617389
     Accuracy 84.76666666666667
     Validation Loss 0.46155629908082224
     Validation Accuracy 83.7
Epoch: 4
     Loss 0.4199296449315399
     Accuracy 85.23333333333333
     Validation Loss 0.4549666638480806
     Validation Accuracy 84.6
Epoch: 5
     Loss 0.41133570723128277
     Accuracy 85.47222222222223
     Validation Loss 0.47035258755132514
     Validation Accuracy 84.36666666666666
Epoch: 6
     Loss 0.40568975497641635
     Accuracy 85.82222222222222
     Validation Loss 0.45181976227351045
     Validation Accuracy 84.98333333333333
Epoch: 7


In [ ]:
import wandb

sweep_config = {
  "name": "Loss_compare",
  "method": "grid",
  "project": "cs6910-assignment01",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "max_epoch": {
            "values":[10]
        },
        "num_hidden": {
            "values":[5]
        }, 
        "size_hidden": {
            "values":[64]
        },
        "weight_decay":{
            "values":[0]
        },
        "learning_rate":{
            "values":[0.001]
        },
        "batch_size": {
            "values":[16]
        },  
        "optimizer": {
            "values":['adam']
        },
        "initializer": {
            "values":['Xavier']
        },
        "activation":{
            "values": ['relu']
        },
        "loss_func":{
            "values": ['mse']
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

# MNIST DATASET

Based on your learnings above, give me 3 recommendations for what would work for the MNIST dataset (not Fashion-MNIST). Just to be clear, I am asking you to take your learnings based on extensive experimentation with one dataset and see if these learnings help on another dataset. If I give you a budget of running only 3 hyperparameter configurations as opposed to the large number of experiments you have run above then which 3 would you use and why. Report the accuracies that you obtain using these 3 configurations.

In [ ]:
def mnist_swp(hyperparameters):
  import wandb
  from keras.datasets import mnist
  from matplotlib import pyplot
  import numpy as np
  import math
  import random
  from sklearn.metrics import confusion_matrix
  import seaborn as sns
  (X_train, y_train), (X_test, y_test) = mnist.load_data()
  X_train_flatten = np.reshape(X_train,(X_train.shape[0],X_train.shape[1]*X_train.shape[2]))/255.0
  X_train_flatten = X_train_flatten.T
  X_test_flatten = np.reshape(X_test,(X_test.shape[0],X_test.shape[1]*X_test.shape[2]))/255.0
  X_test_flatten = X_test_flatten.T
  num_train = X_train.shape[0]
  num_test = X_test.shape[0]




  loss_func = 'cross_entropy'

  wandb.init(project="cs6910-assignment01", config=hyperparameters)
  config = wandb.config
  wandb.run.name = "MNIST_hl{}_bs_{}_ac_{}".format(config.num_hidden,config.batch_size, config.activation)
  L = config.num_hidden+1
  beta1 = 0.9
  beta2 = 0.999
  eps = 1e-8
  gamma = 0.9


  Wb_matrix, update, grad_initial = initialize_Wb_matrix(X_train_flatten, config.num_hidden , config.size_hidden ,config.initializer)
  v = update.copy()
  m_hat = update.copy()
  v_hat = update.copy()

  
  for i in range(config.max_epoch):
    loss = 0
    test_loss = 0
    count = 0
    grads = grad_initial
    print("Epoch:", i+1)
    for j in range(num_train):
      train_ip= np.reshape(X_train_flatten[:,j], (-1, 1)) 
      y_hat, H_caches, A_caches = forward_propagation(train_ip, Wb_matrix ,config.activation)
      if(y_train[j]==np.argmax(y_hat)):
        count = count + 1; 

      if (j+1)%config.batch_size == 0  :
        if config.optimizer == 'nag':
          Wb_matrix, update = nag(Wb_matrix, config.learning_rate,gamma, update, L)
        elif config.optimizer == 'nadam':
          Wb_matrix, update = nag(Wb_matrix, config.learning_rate,gamma, update, L)

      grad = back_propagation(H_caches, A_caches, Wb_matrix, y_hat, [y_train[j]], config.activation, config.weight_decay,loss_func)
      for l in range(L):
        grads['dW' + str(l+1)] = grads['dW' + str(l+1)] + grad['dW' + str(l+1)] 
        grads['db' + str(l+1)] = grads['db' + str(l+1)] + grad['db' + str(l+1)] 


      if (j+1)%config.batch_size == 0 : 
        for l in range(L):
          grads['dW' + str(l+1)] = grads['dW' + str(l+1)]/config.batch_size
          grads['db' + str(l+1)] = grads['db' + str(l+1)]/config.batch_size
        if config.optimizer == 'sgd':
          Wb_matrix = sgd(Wb_matrix, grads, config.learning_rate,L)
        elif config.optimizer == 'momentum':
          Wb_matrix,update = momentum(Wb_matrix,grads,config.learning_rate,gamma,update,L)
        elif config.optimizer == 'nag':
          Wb_matrix,update = momentum(Wb_matrix,grads,config.learning_rate,gamma,update,L)
        elif config.optimizer == 'rmsprop':
          Wb_matrix, update = rmsprop(Wb_matrix, config.learning_rate, update,  beta1, eps, grads, L)
        elif config.optimizer == 'adam':
          Wb_matrix, update,v = adam(Wb_matrix, config.learning_rate, beta1, beta2, update, v, eps, grads, L, i+1)
        elif config.optimizer == 'nadam':
          Wb_matrix, update,v = adam(Wb_matrix, config.learning_rate, beta1, beta2, update, v, eps, grads, L, i+1)    
      
      if loss_func == 'cross_entropy':
        loss = loss - (1/num_train)*math.log(y_hat[y_train[j]])
      elif loss_func == 'mse':
        loss = loss + (1/num_train)*(max(y_hat) - y_train[j])
    
    y_hat_test, H_caches_test, A_caches_test = forward_propagation(X_test, Wb_matrix ,config.activation)
    count_test = np.sum(np.argmax(y_hat_test, axis = 0)== y_test)
    
    for j in range(num_test):
      test_ip= np.reshape(X_test[:,j], (-1, 1)) 
      y_hat_test, H_caches_test, A_caches_test = forward_propagation(test_ip, Wb_matrix ,config.activation)
      if loss_func == 'cross_entropy':
        test_loss = test_loss - (1/num_test)*math.log(y_hat_test[int(y_test[j])])
      elif loss_func == 'mse':
        test_loss = test_loss + (1/num_test)*(max(y_hat_test) - y_test[j])**2
    accuracy = 100*count/num_train
    test_accuracy = 100*count_test/num_test

    print("     Loss", loss)
    print("     Accuracy",accuracy)
    print("     Test Loss", test_loss)
    print("     Test Accuracy", test_accuracy)

    metrics = {'epoch':i, 'test_accuracy': test_accuracy, 'test_loss': test_loss, 'training_accuracy': accuracy, 'training_loss': loss}
    wandb.log(metrics)

  wandb.run.finish()

In [ ]:
hyperparameters1 = dict(
    num_hidden = 5,
    size_hidden = 128,
    weight_decay = 0,
    optimizer = 'adam',
    initializer = 'Xavier',
    activation = 'relu',
    batch_size = 64,
    learning_rate = 0.001,
    max_epoch = 10,
    )
mnist_swp(hyperparameters1)

hyperparameters2 = dict(
    num_hidden = 5,
    size_hidden = 64,
    weight_decay = 0,
    optimizer = 'nag',
    initializer = 'Xavier',
    activation = 'relu',
    batch_size = 16,
    learning_rate = 0.001,
    max_epoch = 10,
    )

mnist_swp(hyperparameters2)

hyperparameters3 = dict(
    num_hidden = 5,
    size_hidden = 128,
    weight_decay = 0.0005,
    optimizer = 'adam',
    initializer = 'Xavier',
    activation = 'relu',
    batch_size = 16,
    learning_rate = 0.000e1,
    max_epoch = 10,
    )

mnist_swp(hyperparameters3)

In [ ]:
import wandb

sweep_config = {
  "name": "MNIST sweep 2",
  "method": "random",
  "project": "cs6910-assignment01_fullsweep",
  "metric":{
      "name":"test_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "max_epoch": {
            "values": [10]
        },
        "num_hidden": {
            "values":[5]
        }, 
        "size_hidden": {
            "values":[128]
        },
        "weight_decay":{
            "values":[0]
        },
        "learning_rate":{
            "values":[0.001]
        },
        "batch_size": {
            "values":[16]
        },  
        "optimizer": {
            "values":['nag']
        },
        "initializer": {
            "values":['Xavier']
        },
        "activation":{
            "values": ['relu']
        },
        "loss_func":{
            "values": ['cross_entropy']
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: g1j26dbr
Sweep URL: https://wandb.ai/karthik21/uncategorized/sweeps/g1j26dbr
